In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

data_dir = "GeneralPlants"

# Define the CNN
class PlantShapeCNN(nn.Module):
    def __init__(self):
        super(PlantShapeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Assuming input images are 64x64
        self.fc2 = nn.Linear(128, 8)  # Assuming 5 plant shape classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset (Example: from a folder)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Initialize model, loss, and optimizer
model = PlantShapeCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

print("Training complete!")


Epoch 1, Loss: 0.7619714551680797
Epoch 2, Loss: 0.33493742306490204
Epoch 3, Loss: 0.2256988828649392
Epoch 4, Loss: 0.1796471700575706
Epoch 5, Loss: 0.11186283784037507
Epoch 6, Loss: 0.09338335329163316
Epoch 7, Loss: 0.049583666869816745
Epoch 8, Loss: 0.05661228354516867
Epoch 9, Loss: 0.033783028207442445
Epoch 10, Loss: 0.01614531534063202
Training complete!


In [13]:
test_data = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

In [14]:
learning_rate = 0.01
classes = ['Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Northern Leaf Blight', 'Milho - Saudavel',
           'Soja - Masaico Amarelo', 'Soja - Sudden Death Syndrone', 'Trigo - Ferrugem Listrada', 'Trigo - Saudavel', 'Trigo - Septoria']

In [15]:
# Coloca o modelo em modo de avaliação
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

correct = 0
total = 0

# Desativando o cálculo de gradiente para economizar memória
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Exibir precisão do modelo
accuracy = 100 * correct / total
print(f'Precisão do modelo no conjunto de teste: {accuracy:.2f}%')

dataiter = iter(test_loader)
images, labels = next(dataiter)

# Movendo para GPU se disponível
images, labels = images.to(device), labels.to(device)

outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Previsões:', [classes[p] for p in predicted])  # Nome das classes previstas
print('Labels reais:', [classes[l] for l in labels])  # Nome das classes corretas


Precisão do modelo no conjunto de teste: 95.27%
Previsões: ['Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot']
Labels reais: ['Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - 

In [16]:
MODEL_PATH = "TrainedGeneralAi/GeneralAi-V01.pth"

# Salvando o modelo treinado
torch.save(model.state_dict(), MODEL_PATH)
print(f"Modelo salvo em: {MODEL_PATH}")

Modelo salvo em: TrainedGeneralAi/GeneralAi-V01.pth


Carregar Modelo Salvo

In [ ]:
# 1. Criar a mesma estrutura da rede neural
model = PlantShapeCNN()

# 2. Carregar os pesos salvos no modelo
model.load_state_dict(torch.load(MODEL_PATH))

# 3. Definir o modelo para modo de avaliação
model.eval()
print("Modelo carregado com sucesso!")